In [1]:
import matplotlib
import sys
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import time

start_time=time.time()
#Ask for spin array length
N=int(input("Please, set the length N of the NxN spin array:"))
#Number of Montecarlo sweeps for equilibration
eqSteps = 100       
# Number of Montecarlo sweeps for calculation
mcSteps = 1000       
# Number of measurements
mcMeasure=mcSteps/10
T = np.arange(1, 3.1, 0.1);
n = 1/(N*N)


#Generate initial state in which all spins have same direction. For Glauber this will be the equilibrium state at low temperature, which is the starting regime 
def initialstate(N):

    initspin=np.ones((N,N), dtype=float)
    return initspin

#Calculate energy for a given spin configuration.
def calcEnergy(spin):

    Totalenergy=0
    
    for i in range(N):
        for j in range(N):
            if i == (N-1):
                if j == (N-1) :
                    energy=spin[i,j]*(spin[0,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1])

                else:

                    if j == 0:
                        energy=-spin[i,0]*(spin[0,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1])

                    else:
                        energy=-spin[i,j]*(spin[0,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1])
            elif i==0:

                if j == (N-1):
                    energy=-spin[0,j]*(spin[0,0]+spin[1,j]+spin[N-1,j]+spin[0,j-1])

                elif j == 0:
                    energy=-spin[0,0]*(spin[0,1]+spin[1,0]+spin[N-1,0]+spin[0,N-1])

                else:
                    energy=-spin[0,j]*(spin[0,j+1]+spin[1,j]+spin[N-1,j]+spin[0,j-1])

            else:

                if j == (N-1) :
                    energy=-spin[i,j]*(spin[i+1,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1])

                elif j == 0:
                    energy=-spin[i,0]*(spin[i+1,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1])

                else:
                    energy=-spin[i,j]*(spin[i+1,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1])

            Totalenergy+=energy
    
    return Totalenergy/2


#Monte carlo using Glauber´s algorithm
def moveGlauber(spin,beta):
    
    for a in range(N):
        for b in range(N):
            i = np.random.randint(0, N)
            j = np.random.randint(0, N)
            newspin =  spin[i, j]
            if i == (N-1):
                if j == (N-1) :
                    sumspins = spin[0,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1]
        
                else:
            
                    if j == 0:
                        sumspins = spin[0,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1]
        
                    else:
                        sumspins = spin[0,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1]
            elif i==0:

                if j == (N-1):
                    sumspins = spin[0,0]+spin[1,j]+spin[N-1,j]+spin[0,j-1]
        
                elif j == 0:
                    sumspins = spin[0,1]+spin[1,0]+spin[N-1,0]+spin[0,N-1]
        
                else:
                    sumspins = spin[0,j+1]+spin[1,j]+spin[N-1,j]+spin[0,j-1]
    
            else:
        
                if j == (N-1) :
                    sumspins = spin[i+1,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1]
        
                elif j == 0:
                    sumspins = spin[i+1,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1]
        
                else:
                    sumspins = spin[i+1,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1]
            
            Ecost = 2*newspin*sumspins
            p= min(1, np.exp(-Ecost*beta))
            randomnum=np.random.rand()
            if randomnum < p :
                newspin *= -1
            spin[i,j] = newspin
    
    return spin

#Calculates magnetization
def calcMag(spin):
    mag = np.sum(spin)
    return mag

Ene=np.zeros((len(T),int(mcMeasure)))
Mag=np.zeros((len(T),int(mcMeasure)))

#Set first spin configuration using initial state function
spin = initialstate(N)   

#Execute code
for t in range(len(T)):
             
    beta=1.0/T[t]
    
    #Equilibrate system
    for i in range(eqSteps):         
        moveGlauber(spin, beta)

    #Take measurements
    for i in range(int(mcMeasure)):
        for j in range(10):
            moveGlauber(spin, beta)
        #Calculate energy and magnetization    
        Ene[t][i] = calcEnergy(spin)     
        Mag[t][i] = calcMag(spin)                      

#Create data files for energy and magnetization    
fenergyGlauber=pd.DataFrame(Ene)
fmagnetGlauber=pd.DataFrame(Mag)


fenergyGlauber.to_csv('fenergyGlauberN'+str(N)+'.csv',index=False)
fmagnetGlauber.to_csv('fmagnetGlauberN'+str(N)+'.csv',index=False)

#Open energy and magnetization data files 
EnergyDat = pd.read_csv('fenergyGlauberN'+str(N)+'.csv')
MagnetDat = pd.read_csv('fmagnetGlauberN'+str(N)+'.csv')

#Compute energy and magnetization means from data files
Eplot= EnergyDat.mean(axis=1)
Mplot= MagnetDat.mean(axis=1)

#Compute squared energy and square magnetization means from data files
E2=np.average(EnergyDat**2,axis=1)
M2=np.average(MagnetDat**2,axis=1)

#Calculate heat capacity and susceptibility
C = (E2 - (Eplot**2) )/((T**2)*(N**2))
X = (M2 - (Mplot**2))/(T*(N**2))

#Create heat capacity and susceptibility data files
fCGlauber=pd.DataFrame(C)
fXGlauber=pd.DataFrame(X)
fCGlauber.to_csv('fCGlauberN'+str(N)+'.csv', index = False)
fXGlauber.to_csv('fXGlauberN'+str(N)+'.csv', index = False)


print("It took: "+str((time.time()-start_time)/60)+" minutes to run")

Please, set the length N of the NxN spin array:6
It took: 0.22224187056223552 minutes to run
